In [1]:
import pandas as pd

In [2]:
all_muts_df = pd.read_pickle("./data/2_5_df.pkl")
display(all_muts_df.shape, all_muts_df.head())

(2641, 33)

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,...,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links
0,2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,...,{},True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{},[],{}
1,6,,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,...,{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK120000854'], 'ECK1200007...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000079...",[],{},"[{'name': 'Pyrimidine metabolism'}, {'name': '...","{'Pyrimidine metabolism': ['ECK120014627'], 'D..."
2,7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,...,{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},"[{'name': 'Purine metabolism'}, {'name': 'Pyri...","{'Purine metabolism': ['ECK120016992'], 'Pyrim..."
3,8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,...,{},True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{'name': 'CRP'}],{'CRP': ['yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC']},[],{}
4,0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,...,{},True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'CRP'}, {'name': 'NagC'}, {'name': '...","{'PhoP': ['nagBAC-umpH'], 'CRP': ['nagBAC-umpH...","[{'name': 'Galactose metabolism'}, {'name': 'A...","{'Galactose metabolism': ['ECK125162049'], 'Am..."


In [3]:
cog_df = pd.read_pickle("./data/COG_df.pkl")
TU_objects_df = pd.read_pickle("./data/TU_objects_df.pkl")
operon_df = pd.read_pickle("./data/operon_df.pkl")
TU_df = pd.read_pickle("./data/TU_df.pkl")
gene_synonym_df = pd.read_pickle("./data/gene_synonym_df.pkl")

!!! Currently using lists instead of sets for COG hits because I want to know the count of items per COG affected with each mutation, where a mutation can affect multiple unique genomic targets that belong to the same COG. If I used a set, would lose the magnitude of hits to a COG per mutations.  
THIS SHOULD GET FIXED WITH COG LINK DICTS, WHERE THE NUMBER OF TIMES AN OPERON SHOWS UP CAN IN THE LINKING LIST CAN BE THE NUMBER OF GENES INVOVLED IN MUTATING THE COG.

In [4]:
from util.gene import get_gene_bnum


# This is currently not attributing gene without COG to "Function unknown"
# Last time checking this, it wasn't necessary to add the addition "Function unknown" entries here
# though these extra entries were required with the link dictionaries
def _get_COGs(gene_id):
    annots = []
    bnum = get_gene_bnum(gene_id, gene_synonym_df)
    gene_cog_df = cog_df[cog_df["locus"] == bnum]
    if len(gene_cog_df) > 0:
        for COG in gene_cog_df["COG description"]:
            d = {"name": COG}
            annots.append(d)
    return annots


def get_COGs(mut_row):
    annots = []
    for feat_d in mut_row["genomic features"]:
        if feat_d["genetic"]:
            annots += _get_COGs(feat_d["RegulonDB ID"])
        # This will catch whether no features exist in the feat_ID_set
        # or if those features are just integenic regions.
        # Get operon genetic feature link dict through overlapping mutation and operon ranges.
        # All genetic features are annotated within both the "genetic features" and "genetic feature links" column.
        # If no integenic genomic feature is hit by a mutation, use the genetic feature range in an overlap analysis with operons.
        else:
            TU_IDs = set(TU_objects_df[TU_objects_df["TU_OBJECT_ID"] == feat_d["RegulonDB ID"]]["TRANSCRIPTION_UNIT_ID"])
            for TU_ID in TU_IDs:
                subset_TU_genes_df = TU_objects_df[(TU_objects_df["TRANSCRIPTION_UNIT_ID"] == TU_ID) & (
                    TU_objects_df["TU_OBJECT_CLASS"] == 'GN')]
                for gene_id in subset_TU_genes_df["TU_OBJECT_ID"].drop_duplicates():
                    # RegulonDB can return 'nan' or '' gene names
                    if str(gene_id) != 'nan' and gene_id != '':
                        annots += _get_COGs(gene_id)
    return annots


test_mut_row_d = {
    "exp": "test exp",
    "genomic features":
    [
        {'name': 'rph-pyrE attenuator terminator',
         'RegulonDB ID': 'ECK125144791',
         'range': (3815799, 3815828),
         'genetic': False,
         'feature type': 'attenuator terminator',
         'operon': 'rph-pyrE',
         'mutation set count': 5,
         'significant': True},
        {'name': 'rph',
         'RegulonDB ID': 'ECK120000854',
         'range': (3815863, 3816549),
         'genetic': True,
         'feature type': 'gene',
         'operon': 'rph-pyrE',
         'mutation set count': 5,
         'significant': True}
    ]}
assert(get_COGs(test_mut_row_d) == [{'name': 'Nucleotide transport and metabolism'}])


all_muts_df["COGs"] = all_muts_df.apply(lambda r: get_COGs(r), axis=1)
all_muts_df.head()

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,...,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links,COGs
0,2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,...,True,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{},[],{},[{'name': 'Lipid transport and metabolism'}]
1,6,,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,...,True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK120000854'], 'ECK1200007...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000079...",[],{},"[{'name': 'Pyrimidine metabolism'}, {'name': '...","{'Pyrimidine metabolism': ['ECK120014627'], 'D...",[{'name': 'Nucleotide transport and metabolism'}]
2,7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,...,True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},"[{'name': 'Purine metabolism'}, {'name': 'Pyri...","{'Purine metabolism': ['ECK120016992'], 'Pyrim...",[{'name': 'Transcription'}]
3,8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,...,True,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{'name': 'CRP'}],{'CRP': ['yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC']},[],{},[{'name': 'Signal transduction mechanisms'}]
4,0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,...,True,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'CRP'}, {'name': 'NagC'}, {'name': '...","{'PhoP': ['nagBAC-umpH'], 'CRP': ['nagBAC-umpH...","[{'name': 'Galactose metabolism'}, {'name': 'A...","{'Galactose metabolism': ['ECK125162049'], 'Am...",[{'name': 'Carbohydrate transport and metaboli...


In [5]:
from util.operon import get_operon_name_set, get_operon_ID_set


def _add_links_to_d(bnum, d, op_ID_set):
    gene_cog_df = cog_df[cog_df["locus"]==bnum]
    if len(gene_cog_df) > 0:
        for cog in gene_cog_df["COG description"]:
            if cog not in d.keys(): d[cog] = []
            d[cog] += list(op_ID_set)
#     else:
#         if "Function unknown" not in d.keys(): d["Function unknown"] = []
#         d["Function unknown"] = list(op_ID_set)


def get_COG_operon_links(mut_row):
    COG_op_link_d = dict()
    for feat_d in mut_row["genomic features"]:

#             link_exists = False

        if feat_d["genetic"]:
            op_ID_set = get_operon_ID_set(feat_d["RegulonDB ID"], TU_objects_df, TU_df, operon_df)
            if str(feat_d["RegulonDB ID"]) != 'nan' and feat_d["RegulonDB ID"] != '':
                bnum = get_gene_bnum(feat_d["RegulonDB ID"], gene_synonym_df)
                _add_links_to_d(bnum, COG_op_link_d, op_ID_set)
#                     link_exists = True
        else:
            TU_IDs = set(TU_objects_df[TU_objects_df["TU_OBJECT_ID"]==feat_d["RegulonDB ID"]]["TRANSCRIPTION_UNIT_ID"])
            for TU_ID in TU_IDs:
                subset_TU_genes_df = TU_objects_df[(TU_objects_df["TRANSCRIPTION_UNIT_ID"]==TU_ID) & (TU_objects_df["TU_OBJECT_CLASS"]=='GN')]

                op_ID_set = set()
                for _, TU_gene in subset_TU_genes_df.iterrows():
                     op_ID_set = op_ID_set | get_operon_ID_set(TU_gene["TU_OBJECT_ID"], TU_objects_df, TU_df, operon_df)

                for gene_id in subset_TU_genes_df["TU_OBJECT_ID"].drop_duplicates():
                    if str(gene_id) != 'nan' and gene_id != '':  # RegulonDB can return 'nan' or '' gene names
                        bnum = get_gene_bnum(gene_id, gene_synonym_df)
                        _add_links_to_d(bnum, COG_op_link_d, op_ID_set)
#                             link_exists = True

        # Per mutation feature
        # Need to be looking at previously populated level of annotations depdending on how the current level is being populated
        # for regulon, it just means that if there are no operons, go immediately to genomic features.
#             if link_exists == False:
#                 if "Function unknown" not in COG_op_link_d.keys(): COG_op_link_d["Function unknown"] = []
#                 if len(operon_name_set) > 0: # need to always check the nearest annotation type for defined features
#                     COG_op_link_d["Function unknown"] += list(operon_name_set)
#                 else:
#                     COG_op_link_d["Function unknown"].append(feat_d["RegulonDB ID"])

    return COG_op_link_d


all_muts_df["COG links"] = all_muts_df.apply(get_COG_operon_links, axis=1)
all_muts_df.head()

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,...,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links,COGs,COG links
0,2,R110G (CGT→GGT),clsA,SNP,1308318,NaN,G→C,1,42C,124,...,"[{'name': 'clsA', 'RegulonDB ID': 'ECK12000155...",{'ECK120001556': ['ECK120001556']},"[{'name': 'clsA-yciU', 'RegulonDB ID': 'ECK125...",{'ECK125147415': ['ECK120001556']},[],{},[],{},[{'name': 'Lipid transport and metabolism'}],{'Lipid transport and metabolism': ['ECK125147...
1,6,,rph,DEL,3815859,NaN,Δ82 bp,1,42C,124,...,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK120000854'], 'ECK1200007...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000079...",[],{},"[{'name': 'Pyrimidine metabolism'}, {'name': '...","{'Pyrimidine metabolism': ['ECK120014627'], 'D...",[{'name': 'Nucleotide transport and metabolism'}],{'Nucleotide transport and metabolism': ['ECK1...
2,7,A734V (GCG→GTG),rpoC,SNP,4187550,NaN,C→T,1,42C,124,...,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},"[{'name': 'Purine metabolism'}, {'name': 'Pyri...","{'Purine metabolism': ['ECK120016992'], 'Pyrim...",[{'name': 'Transcription'}],{'Transcription': ['ECK120016992']}
3,8,D9A (GAT→GCT),hfq,SNP,4400313,NaN,A→C,1,42C,124,...,"[{'name': 'hfq', 'RegulonDB ID': 'ECK120000431...",{'ECK120000431': ['ECK120000431']},[{'name': 'yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC...,{'ECK125147410': ['ECK120000431']},[{'name': 'CRP'}],{'CRP': ['yjeF-tsaE-amiB-mutL-miaA-hfq-hflXKC']},[],{},[{'name': 'Signal transduction mechanisms'}],{'Signal transduction mechanisms': ['ECK125147...
4,0,coding (380‑400/1149 nt),nagA,DEL,702352,NaN,Δ21 bp,1,42C,124,...,"[{'name': 'nagA', 'RegulonDB ID': 'ECK12000062...",{'ECK120000625': ['ECK120000625']},"[{'name': 'nagBAC-umpH', 'RegulonDB ID': 'ECK1...",{'ECK125162049': ['ECK120000625']},"[{'name': 'CRP'}, {'name': 'NagC'}, {'name': '...","{'PhoP': ['nagBAC-umpH'], 'CRP': ['nagBAC-umpH...","[{'name': 'Galactose metabolism'}, {'name': 'A...","{'Galactose metabolism': ['ECK125162049'], 'Am...",[{'name': 'Carbohydrate transport and metaboli...,{'Carbohydrate transport and metabolism': ['EC...


In [6]:
all_muts_df.to_pickle("./data/2_6_df.pkl")